In [2]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
data = open('./data/input.txt','r')
data = list(data)
data_use = []
data_f = []
index = 0
temp = ''
for d in data:
    if d == "<start>\r\n":
        index = 1
        continue
    if d == '<end>\r\n':
        index = 0
    if index == 1:
        temp = temp + str(d)
    if index == 0:
        data_use.append(temp)
        temp = ''
#print(d)
for d in data_use:
    if len(d) > 40:
        data_f.append(d)
data_use = data_f
all_character = set()
counter = 0
for d in data_use:
    for dd in d:
        all_character.add(dd)
all_character = list(all_character)
nletter = len(all_character)

In [4]:
print(data_use[2])

XIV?me si?cle)
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-03-27
Z:Pour toute observation mailto:galouvielle@free.fr
M:2/2
L:1/4
Q:1/4=110
K:Fmaj
V:1
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G2 zG|
w:Stel-__la splen-dens in mon-____te ut so-lis ra-_di-um Mi-__ra-cu-lis ser-ra-____to ex-au-di po-_pu-__lum  Prin-
Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 zG|Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 z2||
w:ci-pes et ma-gna-__ tes ex-stir-pe re-_gi-_a sae-cu-li po-tes-ta-__tes ob-ten-ta ve-_ni-_a
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G4|]
w:Pe-__cca mi num pro-cla-____mant tu den tes pec-_to-ra Po-__pli te fle-xo cla-____mant hic A-ve Ma-_ri-__a
V:2
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d2 zd-|
d G2 G|cB A2|FG Bc|d2 zd-|d G2 G|cB A2|FG Bc|d2 z2|
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d4|]



In [5]:
def lineToTensor(line):
    tensor = torch.zeros(len(line),nletter)
    position = np.zeros(len(line))
    for i, letter in enumerate(line):
        position[i] = all_character.index(letter)
        tensor[i][all_character.index(letter)] = 1
    position = torch.LongTensor(position[1:])
    return autograd.Variable(tensor[:-1,:]).cuda(),autograd.Variable(position).cuda()

In [6]:
def batch_g(data,batch_size):
    lyric = np.random.randint(len(data))
    #print(len(data[lyric]))
    if len(data[lyric])-batch_size >0:
        interval_s = np.random.randint(len(data[lyric])-batch_size)
        return data[lyric][interval_s:interval_s+batch_size]
    else:
        return []

In [7]:
class LSTM(nn.Module):
    def __init__(self,letter_dim,hidden_dim):
        super(LSTM, self).__init__()
        self.letter_dim = letter_dim
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
        
        self.lstm = nn.LSTM(letter_dim,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,letter_dim)
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
    def forward(self,line):
        lstm_out, self.hidden = self.lstm(line.view(len(line),1,-1), self.hidden)
        out_space = self.hidden2out(lstm_out.view(len(lstm_out),-1))
#         out_scores = F.softmax(out_space / 1.0,dim = 1)
        return out_space

In [8]:
hidden_dim = 100
batch_size = 30
lstm = LSTM(nletter,hidden_dim).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.001)

In [9]:
for epoch in range(1000):
    lstm.zero_grad()
    batch = batch_g(data_use,batch_size)
#     print(batch)
#     print('\n')
    if len(batch)>0:
        batch_in,batch_t = lineToTensor(batch)
        lstm.hidden = lstm.init_hidden()
        out_scores = lstm(batch_in)
        loss = loss_function(out_scores,batch_t)
        if epoch % 100 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

Variable containing:
 4.5327
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.3036
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.9116
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.8929
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.0180
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.0230
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.3382
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.7838
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.9355
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.5177
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [10]:
print(F.softmax(out_scores / 1.0,dim = 1).data.cpu().numpy()[1])

[  1.54758021e-01   5.49156778e-03   5.47980051e-03   3.61364661e-03
   5.90475556e-03   2.92981439e-03   4.23212186e-04   5.27554948e-04
   1.20396484e-02   6.94739749e-04   2.17154948e-03   1.26677728e-03
   2.03057961e-03   1.58910079e-05   1.15190365e-03   5.82225733e-02
   4.63690422e-03   4.93470393e-03   2.06679595e-03   5.00158733e-03
   7.87919620e-04   6.03018813e-02   6.46401255e-04   2.04229844e-03
   8.87830829e-05   3.77629809e-02   2.20712088e-02   2.25286488e-03
   8.08291021e-04   4.04857984e-03   4.58172448e-02   8.96572717e-04
   8.50588491e-04   1.32862269e-03   4.97767527e-04   5.46403520e-04
   3.24365072e-04   4.98249717e-02   2.34470144e-02   1.58840325e-03
   6.00826740e-03   3.62070277e-03   6.56193413e-04   2.52603586e-05
   1.05222156e-02   6.82847342e-03   2.11721035e-05   2.10184444e-05
   1.67630706e-03   5.24589047e-02   2.69264029e-03   1.21448627e-02
   9.93109960e-03   6.39499128e-02   2.90866923e-02   5.16015047e-04
   6.12060772e-04   1.02132454e-03

In [11]:
present = torch.zeros(1,nletter)
present[0][all_character.index('X')] = 1
present = autograd.Variable(present).cuda()
lyric = []
lyric.append(all_character[int(torch.distributions.Categorical(F.softmax(lstm(present)/1.0,dim = 1).data.cpu()).sample().numpy())])
for num_generation in range(400):
    #print(lstm.forward(present).cpu())
    next_char_index = int(torch.distributions.Categorical(F.softmax(lstm(present)/1.0,dim = 1).data.cpu()).sample().numpy())
    next_char = all_character[next_char_index]
    lyric.append(next_char)
    present = torch.zeros(1,nletter)
    present[0][all_character.index(next_char)] = 1
    present = autograd.Variable(present).cuda()

In [12]:
print(str(lyric))

['G', 'c', 'f', ' ', 'A', ' ', 'G', '2', 'B', '2', ' ', '>', '2', 'A', 'f', 'B', 'F', 'd', 'B', ' ', ':', 'P', 'e', 'D', 'E', '/', 'B', 'F', 'e', 'A', 'D', 'h', '|', 'e', 'B', ' ', 'e', ' ', 'E', 'e', 'c', 'G', 'B', '2', 'G', 'A', '|', '|', 'B', 'F', 'c', 'e', 'G', '|', 'A', '2', '|', '2', 'G', 'c', '2', 'A', ' ', 'B', 'c', 'F', 'c', '2', 'c', 'd', 'A', '"', '3', ' ', 'B', '|', 'e', 'A', 'G', 'h', ' ', 'A', '2', 'c', 'A', '|', ' ', 'c', ' ', '|', ' ', 'E', '4', 'G', '\n', '4', 'G', '|', 'd', 'c', 'A', 'A', 'c', '|', '2', ' ', 'B', ' ', ' ', 'B', 'd', 'G', 'e', '|', 'A', 'B', 'F', '|', 'e', 'B', 'E', '|', 'E', '=', 'c', 'd', 'B', 'B', '3', '1', 'e', 'B', '/', 'G', 'G', '2', 'd', ' ', 'B', '|', ' ', 'B', 'H', 'G', 'd', '|', ' ', 'G', 'B', 'd', 'A', '|', '|', '|', '|', '|', 'F', 'c', 'B', 'f', 'f', 'd', 'Z', 'F', 'd', 'A', 'c', 'A', 'A', 'G', ' ', 'g', 'B', ',', '|', ' ', 'F', 'e', 'B', 'B', 'f', '|', '"', ' ', 'A', 'B', '|', '2', 'c', '2', 'c', 'A', '|', 'g', 'B', '\n', 'd', ' ', 'A', 'G

In [13]:
Music = ""
for char in lyric:
    Music = Music + char
print(Music)

Gcf A G2B2 >2AfBFdB :PeDE/BFeADh|eB e EecGB2GA||BFceG|A2|2Gc2A BcFc2cdA"3 B|eAGh A2cA| c | E4G
4G|dcAAc|2 B  BdGe|ABF|eBE|E=cdBB31eB/GG2d B| BHGd| GBdA|||||FcBffdZFdAcAAG gB,| FeBBf|" AB|2c2cA|gB
d AGc-e:ccf~4AGc2BD Add2A 'c:d|A|G|23|AA3AfcAc Gdf A26:FA:^bd:BgEEBA/ cGA3 AF|AaBD  c /ae3 BaF :F dGg f2gDd|c| d|3A|B A| D2cB|d|e~G|cFG!|d 2 A dfA2 B||  A\F~| dogc| A ef2EB 3A|c2f:AABFGAF|1f2|:|2dA2FdGGA a
